# Example: Cross-contour transport

This notebook uses the `cosima_cookbook` to calculate transport across an arbitrary contour. We do this by first creating the contour, such as sea surface height, and extracting the coordinates using `matplotlib`'s `Path` class. We then create some masks to indicate which direction is across the contour at each position along the contour. We then load the transport data and compute the transport, resulting in data with dimensions depth and along contour index. 

Computation times shown used conda environment `analysis3-22.07` on the large compute size on NCI's ARE.

First, we load useful packages:

In [1]:
%matplotlib inline

import cosima_cookbook as cc
import matplotlib.pyplot as plt
import netCDF4 as nc
import xarray as xr
import numpy as np
import glob
import os
import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)
logging.getLogger('distributed.utils_perf').setLevel(logging.ERROR)
from dask.distributed import Client

In [2]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 28,Total memory: 250.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40253,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 250.00 GiB
Comm: tcp://127.0.0.1:40493,Total threads: 4
Dashboard: /proxy/36807/status,Memory: 35.71 GiB
Nanny: tcp://127.0.0.1:45147,


#### Choose database

In [3]:
session = cc.database.create_session()

#### Choose experiment

In [4]:
experiment = '01deg_jra55v140_iaf_cycle3'

#### Choose a latitude range so the contour fits in the range, but there is not too much extra space. Extra space slows down the computation.

In [5]:
lat_range = slice(-79, -59)

In [20]:
#%%time
#importing all heat transports calculated offline
# src_HT='/g/data/x77/wf4500/ASC_project/model_data/access-om2/01deg_jra55v13_ryf9091/daily_heat_transport'




# glx =glob.glob(src_HT + "/temp_xflux_from_daily_*******.nc")
# glx=sorted(glx)
# print("%i netcdf data files"%len(glx))
# temp_xflux=xr.open_mfdataset(glx, concat_dim='time', combine='nested')


# gly =glob.glob(src_HT + "/temp_yflux_from_daily_*******.nc")
# gly=sorted(gly)
# print("%i netcdf data files"%len(gly))
# temp_yflux=xr.open_mfdataset(gly, concat_dim='time', combine='nested')
# temp_yflux = temp_yflux.sel(time=slice('2170-03','2170-03'))
# test.time

<xarray.DataArray 'time' (time: 31)>
array([cftime.DatetimeNoLeap(2170, 3, 1, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 2, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 3, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 4, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 5, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 6, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 7, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 8, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 9, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 10, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 11, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 12, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 13, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 14, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 15, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 16, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 17, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 18, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 19, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 20, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 21, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 22, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 23, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 24, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 25, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 26, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 27, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 28, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 29, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 30, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 3, 31, 12, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 2170-03-01 12:00:00 ... 2170-03-31 12:00:00
Attributes:
    long_name:       time
    cartesian_axis:  T
    calendar_type:   NOLEAP
    bounds:          time_bounds

In [8]:
# start_time='2170-01-01'
# yt_ocean = cc.querying.getvar('01deg_jra55v13_ryf9091','yt_ocean',start_time=start_time, end_time=end_time)

In [19]:
int(1)

1

In [21]:
vhrho = cc.querying.getvar(experiment,'vhrho_nt',session,ncfile='%daily%',n=-1)
vhrho.time

<xarray.DataArray 'time' (time: 31)>
array(['2018-12-01T12:00:00.000000000', '2018-12-02T12:00:00.000000000',
       '2018-12-03T12:00:00.000000000', '2018-12-04T12:00:00.000000000',
       '2018-12-05T12:00:00.000000000', '2018-12-06T12:00:00.000000000',
       '2018-12-07T12:00:00.000000000', '2018-12-08T12:00:00.000000000',
       '2018-12-09T12:00:00.000000000', '2018-12-10T12:00:00.000000000',
       '2018-12-11T12:00:00.000000000', '2018-12-12T12:00:00.000000000',
       '2018-12-13T12:00:00.000000000', '2018-12-14T12:00:00.000000000',
       '2018-12-15T12:00:00.000000000', '2018-12-16T12:00:00.000000000',
       '2018-12-17T12:00:00.000000000', '2018-12-18T12:00:00.000000000',
       '2018-12-19T12:00:00.000000000', '2018-12-20T12:00:00.000000000',
       '2018-12-21T12:00:00.000000000', '2018-12-22T12:00:00.000000000',
       '2018-12-23T12:00:00.000000000', '2018-12-24T12:00:00.000000000',
       '2018-12-25T12:00:00.000000000', '2018-12-26T12:00:00.000000000',
       '2018-12-27T12:00:00.000000000', '2018-12-28T12:00:00.000000000',
       '2018-12-29T12:00:00.000000000', '2018-12-30T12:00:00.000000000',
       '2018-12-31T12:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2018-12-01T12:00:00 ... 2018-12-31T12:00:00
Attributes:
    long_name:       time
    cartesian_axis:  T
    calendar_type:   GREGORIAN
    bounds:          time_bounds

In [45]:
experiment = '01deg_jra55v140_iaf_cycle3'
start_time='2007-01-01'
end_time='2007-01-31'
temp_yflux = cc.querying.getvar(experiment,'temp_yflux_adv',session,start_time=start_time, end_time=end_time).sel(time=slice(start_time, end_time))
u = cc.querying.getvar(experiment,'u',session,start_time=start_time, end_time=end_time).sel(time=slice(start_time, end_time))
u


<xarray.DataArray 'u' (time: 1, st_ocean: 75, yu_ocean: 2700, xu_ocean: 3600)>
dask.array<getitem, shape=(1, 75, 2700, 3600), dtype=float32, chunksize=(1, 19, 135, 180), chunktype=numpy.ndarray>
Coordinates:
  * xu_ocean  (xu_ocean) float64 -279.9 -279.8 -279.7 -279.6 ... 79.8 79.9 80.0
  * yu_ocean  (yu_ocean) float64 -81.09 -81.05 -81.0 -80.96 ... 89.92 89.96 90.0
  * st_ocean  (st_ocean) float64 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
  * time      (time) datetime64[ns] 2007-01-16T12:00:00
Attributes: (12/13)
    long_name:      i-current
    units:          m/sec
    valid_range:    [-10.  10.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_c geolat_c
    ...             ...
    ncfiles:        ['/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_...
    contact:        Andrew Kiss
    email:          andrew.kiss@anu.edu.au
    created:        2020-11-11
    description:    0.1 degree ACCESS-OM2 global model configuration under in...
    notes:          Run configuration and history: https://github.com/COSIMA/...

In [44]:
#experiment='01deg_jra55v13_ryf9091'
experiment = '01deg_jra55v140_iaf_cycle3'
hu = cc.querying.getvar(experiment, 'hu', session, n=1)

# Import grid cell length
dxu = cc.querying.getvar(experiment, 'dxu', session, n=1)
dxu=dxu.rename({'xu_ocean':'ni','yu_ocean':'nj'})
dyu = cc.querying.getvar(experiment, 'dyu', session, n=1)
dyu=dyu.rename({'xu_ocean':'ni','yu_ocean':'nj'})
dxu

<xarray.DataArray 'dxu' (nj: 2700, ni: 3600)>
dask.array<open_dataset-2e557d26d286422c684335dac3891350dxu, shape=(2700, 3600), dtype=float32, chunksize=(540, 720), chunktype=numpy.ndarray>
Coordinates:
  * ni       (ni) float64 -279.9 -279.8 -279.7 -279.6 ... 79.7 79.8 79.9 80.0
  * nj       (nj) float64 -81.09 -81.05 -81.0 -80.96 ... 89.87 89.92 89.96 90.0
Attributes:
    long_name:     ocean dxu on u-cells
    units:         m
    valid_range:   [-1.e+09  1.e+09]
    cell_methods:  time: point
    coordinates:   geolon_c geolat_c
    ncfiles:       ['/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_j...
    contact:       Andrew Kiss
    email:         andrew.kiss@anu.edu.au
    created:       2020-11-11
    description:   0.1 degree ACCESS-OM2 global model configuration under int...
    notes:         Run configuration and history: https://github.com/COSIMA/0...

#### We must make sure that this latitude range is so that the t-cells are always south and west of the u-cells.

This is important because the meridional and zonal transports occur on different grids to each other. We can check this by loading the `u`-cell and `t`-cell coordinates. 

We choose this convention so that later on when we create `numpy` grids of where the contour is and in what direction the contour goes.

In [8]:
%%time

yt_ocean = cc.querying.getvar(experiment, 'yt_ocean', session, n=1)
yt_ocean = yt_ocean.sel(yt_ocean = lat_range)

xt_ocean = cc.querying.getvar(experiment, 'xt_ocean', session, n=1)

yu_ocean = cc.querying.getvar(experiment, 'yu_ocean', session, n=1)
yu_ocean = yu_ocean.sel(yu_ocean = lat_range)

xu_ocean = cc.querying.getvar(experiment, 'xu_ocean', session, n=1)

CPU times: user 45.2 s, sys: 22.6 s, total: 1min 7s
Wall time: 53.4 s


#### Load quantity we want a contour of, in our case, Bathymetry

In [11]:
start_time = '2170-01-01'
end_time = '2170-12-31'
time_slice = slice(start_time, end_time)
bath = cc.querying.getvar(experiment,'ht', session, n=-1)

# select one year and latitude range
bath_mean = bath.sel(yt_ocean = lat_range)

# weighted time-mean by month length
#days_in_month = ssh.time.dt.days_in_month
#days_in_year = 365
#ssh_mean = (ssh * days_in_month / days_in_year).sum('time')
bath_mean

<xarray.DataArray 'ht' (yt_ocean: 516, xt_ocean: 3600)>
dask.array<getitem, shape=(516, 3600), dtype=float32, chunksize=(490, 720), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 -79.0 -78.95 -78.91 ... -56.14 -56.08 -56.03
Attributes:
    long_name:      ocean depth on t-cells
    units:          m
    valid_range:    [-1.e+09  1.e+09]
    cell_methods:   time: point
    coordinates:    geolon_t geolat_t
    standard_name:  sea_floor_depth_below_geoid
    ncfiles:        ['/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_...
    contact:        Andrew Kiss
    email:          andrew.kiss@anu.edu.au
    created:        2020-11-11
    description:    0.1 degree ACCESS-OM2 global model configuration under in...
    notes:          Run configuration and history: https://github.com/COSIMA/...

#### Choose your desired contour value

In [12]:
contour_depth = 1000